In [22]:
import mlflow 
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor

mlflow.set_tracking_uri("http://127.0.0.1:8080") 



LIst of Experiment objects. 

In [2]:
all_experiments = client.search_experiments()
print(all_experiments)

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1758464177436, experiment_id='0', last_update_time=1758464177436, lifecycle_stage='active', name='Default', tags={}>]


In [4]:
default_experiment = [
    {"name": experiment.name, "lifecycle_stage": experiment.lifecycle_stage} 
    for experiment in all_experiments
    if experiment.name == "Default"
][0]

print(default_experiment)

{'name': 'Default', 'lifecycle_stage': 'active'}


In [8]:
experiment_description = (
    "This is  the grocery forecating project"
    "this experimennt contais the produce models for apples"

)
experiment_tags = {
    "project_name" : "grocery forecasting",
    "store_dept" : "produce",
    "team" : "stores-ml",

    "project_quater": "q2-2024",
    "mlflow.note.content" : "Initial experiment with RandomForestRegressor"

}

produce_apples_experiment = client.create_experiment(
    name = "produce-apples",
    tags=experiment_tags
)

In [15]:
apples_experiment = client.search_experiments(
 filter_string="tags.`project_name` = 'grocery forecasting' AND name = 'produce-apples'"
) 

print(apples_experiment[0])

<Experiment: artifact_location='mlflow-artifacts:/341584554361460388', creation_time=1758465576159, experiment_id='341584554361460388', last_update_time=1758465576159, lifecycle_stage='active', name='produce-apples', tags={'mlflow.experimentKind': 'custom_model_development',
 'mlflow.note.content': 'Initial experiment with RandomForestRegressor',
 'project_name': 'grocery forecasting',
 'project_quater': 'q2-2024',
 'store_dept': 'produce',
 'team': 'stores-ml'}>


In [16]:
import  pandas as pd
import numpy as np
from datetime import datetime , timedelta
 

In [19]:
def generate_apple_sales_data_with_promo_adjustment (
        base_demand: int = 1000 , n_rows: int = 5000
):
    np.random.seed(9999)
    dates = [datetime.now()-timedelta(days=i) for i in range(n_rows)]
    dates.reverse()

    df = pd.DataFrame({
        "date" : dates,
        "average_temperature": np.random.uniform(10, 35, n_rows),
            "rainfall": np.random.exponential(5, n_rows),
            "weekend": [(date.weekday() >= 5) * 1 for date in dates],
            "holiday": np.random.choice([0, 1], n_rows, p=[0.97, 0.03]),
            "price_per_kg": np.random.uniform(0.5, 3, n_rows),
            "month": [date.month for date in dates],
        }
    )

    df["inflation_multiplier"] = (
        1 + (df["date"].dt.year - df["date"].dt.year.min()) * 0.03
    )

    # Incorporate seasonality due to apple harvests
    df["harvest_effect"] = np.sin(2 * np.pi * (df["month"] - 3) / 12) + np.sin(
        2 * np.pi * (df["month"] - 9) / 12
    )

    # Modify the price_per_kg based on harvest effect
    df["price_per_kg"] = df["price_per_kg"] - df["harvest_effect"] * 0.5

    # Adjust promo periods to coincide with periods lagging peak harvest by 1 month
    peak_months = [4, 10]  # months following the peak availability
    df["promo"] = np.where(
        df["month"].isin(peak_months),
        1,
        np.random.choice([0, 1], n_rows, p=[0.85, 0.15]),
    )

    # Generate target variable based on features
    base_price_effect = -df["price_per_kg"] * 50
    seasonality_effect = df["harvest_effect"] * 50
    promo_effect = df["promo"] * 200

    df["demand"] = (
        base_demand
        + base_price_effect
        + seasonality_effect
        + promo_effect
        + df["weekend"] * 300
        + np.random.normal(0, 50, n_rows)
    ) * df[
        "inflation_multiplier"
    ]  # adding random noise

    # Add previous day's demand
    df["previous_days_demand"] = df["demand"].shift(1)
    df["previous_days_demand"].fillna(
        method="bfill", inplace=True
    )  # fill the first row

    # Drop temporary columns
    df.drop(columns=["inflation_multiplier", "harvest_effect", "month"], inplace=True)

    return df

In [21]:
data = generate_apple_sales_data_with_promo_adjustment(base_demand=1_000, n_rows=1_000)
data[-20:]

/tmp/ipykernel_16365/1911709837.py:57: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["previous_days_demand"].fillna(
/tmp/ipykernel_16365/1911709837.py:57: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["previous_days_demand"].fillna(


,date,average_temperature,rainfall,weekend,holiday,price_per_kg,promo,demand,previous_days_demand
980,2025-09-02 20:42:35.411888,34.130183,1.454065,0,0,1.449177,0,999.306290,1029.418398
981,2025-09-03 20:42:35.411886,32.353643,9.462859,0,0,2.856503,0,842.129427,999.306290
982,2025-09-04 20:42:35.411883,18.816833,0.391470,0,0,1.326429,0,990.616709,842.129427
983,2025-09-05 20:42:35.411827,34.533012,2.120477,0,0,0.970131,0,1068.802075,990.616709
984,2025-09-06 20:42:35.411825,23.057202,2.365705,1,0,1.049931,0,1346.486305,1068.802075
985,2025-09-07 20:42:35.411823,34.810165,3.089005,1,0,2.035149,0,1329.564672,1346.486305
986,2025-09-08 20:42:35.411821,29.208905,3.673292,0,0,2.518098,0,1086.143402,1329.564672
987,2025-09-09 20:42:35.411819,16.428676,4.077782,0,0,1.268979,0,1093.207186,1086.143402
988,2025-09-10 20:42:35.411817,32.067512,2.734454,0,0,0.762317,0,1069.939894,1093.207186
989,2025-09-11 20:42:35.411815,31.938203,13.883486,0,0,1.153301,0,994.409540,1069.939894


In [24]:
artifact_path = "produce_apples_model"

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score , root_mean_squared_error ,mean_absolute_error
X= data.drop(columns=["date", "demand"])
y = data["demand"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

params = {
    "n_estimators": 100,
    "max_depth": 6,
    "min_samples_split": 10,
    "min_samples_leaf": 4,
    "bootstrap": True,
    "oob_score": False,
    "random_state": 888,
}
rf = RandomForestRegressor(**params)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

metric = { "mae": mae, "mse": mse, "rmse": rmse, "r2": r2 }

with mlflow.start_run(experiment_id=produce_apples_experiment) as run:
    mlflow.log_params(params)
    mlflow.log_metrics(metric)
    mlflow.sklearn.log_model(sk_model=rf,input_example =X_test, name=artifact_path)


/home/seven/Internship/venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🏃 View run angry-trout-306 at: http://127.0.0.1:8080/#/experiments/341584554361460388/runs/3eb0597e30d242b4b4af9cfc4137046f
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/341584554361460388
